# Investigating the connection between phonon DOS and inelastic scattering kernels

In this section we will take a closer look upon the connection between phonon VDOS curves and scattering kernels, using the VDOS to kernel expansion capability of NCrystal (a fast implementation of the method of Sjölander[1]).

*[1] Sjölander, Alf. "Multi-phonon processes in slow neutron scattering by crystals." Arkiv Fysik 14 (1958).*

## Preamble ##
Fix dependencies and tune jupyter a bit. Feel free to replace as you wish:

In [1]:
import pathlib
import os
if 'TSL_SCHOOL_DIR' in os.environ:
    if any( (p/".git").is_dir() for p in (pathlib.Path(".").absolute().resolve()/"dummy").parents ):
        raise RuntimeError('Please copy notebook to a work directory')

In [2]:
#Uncomment to get dependencies via pip: !pip install --quiet ipympl numpy matplotlib

In [3]:
%matplotlib ipympl

In [4]:
import matplotlib
matplotlib.rcParams.update({"figure.figsize":(6.4*0.5,4.8*0.5),"figure.dpi":150,'font.size':5})

In [5]:
%%html
<style>div.jupyter-widgets.widget-label {display: none;}</style>

Always import NCrystal of course:

In [6]:
import NCrystal as NC
assert NC.version_num >=  3005081
NC.test() #< quick unit test that installation works!

Tests completed succesfully


# VDOS to scattering kernel (solids)
As advertised, NCrystal is able to perform a very fast expansion of 1D VDOS curves into 2D scattering kernels when loading a material. This makes input files smaller, and avoids pre-defined temperature choices for the material. Here we will bring out a few formulas, and play around with such an expansion.

## A bit of theory
*(NB: formulas here were not carefully checked for sign errors, factors of 2, etc.)*

Sjölander gives us a recipe for constructing functions $G_n$, and then for getting the scattering kernel as:

* $S(\alpha,\beta) = (4{\pi}kT)\,e^{-2W}\,\sum_{n=1}^\infty\frac{(2W)^n}{n!}G_n(-\beta)$    $\,\,\,\,$(eq. 1)

Here $\delta^2$ is the mean-squared-displacement, and the $\alpha$-dependency comes in via:

* $2W=\delta^2q^2=\left[\frac{\hbar^2}{2km_n}\delta^2\right]\alpha$

If $\rho_\text{DOS}(E)$ is the VDOS, then the $G_1$ function is defined as (up to a normalisation factor):

* $G_1\big(\beta=\frac{E}{{k}T}\big) = \frac{1}{E}\,\rho_\text{DOS}\,\big(|E|\big)\big(\coth(E/2kT)-1\big)$

The factor inside the last parenthesis is essentially a Boltzmann factor providing *detailed balance*, reflecting that the phonon state population at a given energy depends on the temperature. Higher order $G_n$ functions are defined by repeated convolution of the state:

* $G_2=G_1{\circledast}G_1$
* $G_3=G_1{\circledast}G_1{\circledast}G_1{\circledast}G_1$
* $G_4=G_1{\circledast}G_1{\circledast}G_1{\circledast}G_1$
* $\ldots$
* $G_n=G_1{\circledast}\ldots{\circledast}G_1$

In practice NCrystal uses a fast fourier transform (FFT) algorithm (with a few custom tricks to make it very scalable and fast), and does not do $n$ convolution to get to order $n$, instead using e.g. $G_{100}=G_{50}{\circledast}G_{50}$. Additionally, due to the enormous variation in size of the various quantities, utmost care concerning numerical precision must be taken at all steps along the way, from construction of the $G_1$, to the application of (eq. 1).

## A bit of practice
Just a few interactive handles to see how it all comes together. Just play around!

**Disclaimer:** The code below was thrown together last minute, and it suffers from flickering, lack of caching, lack of proper colour-maps, and a generally bad interface. Be gentle!

In [7]:
#DONT LOOK AT THIS CODE, JUST HIDE IT AND PLAY WITH THE RESULTING WIDGET
from ipywidgets import interact
import NCrystal.plot
@interact(order=(0,50),max_order=(0,200),vdoslux=(0,5),
          choice=['VDOS','Gn','S(alpha,beta)','cross sections'],
          cfgstr=['Al_sg225.ncmat',
                  'H in Polyethylene_CH2.ncmat',
                  'C in Polyethylene_CH2.ncmat',
                  'Ca in CaSiO3_sg2_Wollastonite.ncmat'],
          neutron_ekin = '1.8Å',
          temperature=(2.0,1500.0),
          auto_update=True)
def show(order=0,max_order=0,vdoslux=3,choice='',cfgstr='',neutron_ekin='',temperature=296.,auto_update=True):
    if not auto_update:
        return
    import matplotlib.pyplot as plt
    plt.close()
    p=cfgstr.split(' in ')
    info = NC.createInfo(p[-1]+f';temp={temperature}')
    di = info.findDynInfo(p[0]) if len(p)==2 else info.dyninfos[0]
    ne = None
    for u in (('meV',lambda x:0.001*x),('Å',lambda x:NC.wl2ekin(x)),('angstrom',lambda x:NC.wl2ekin(x)),('eV',lambda x:x)):
        if neutron_ekin.endswith(u[0]):
            ne = [u[1](float(neutron_ekin[0:-len(u[0])]))]
            break
    if ne is None:
        print("WARNING: Not able to parse neutron_ekin (allowed units are meV, eV, Å, and angstrom)")
        return
    max_order = max(order,max_order)
    if choice=='VDOS':
        di.plot_vdos()
    elif choice=='Gn':
        di.plot_Gn(**(dict(n=1,nmax=10) if order==0 else dict(n=order,nmax=(max_order if max_order>order else None))))
    elif choice=='S(alpha,beta)':
        knlargs=dict(plot=True,
                     vdoslux=vdoslux,
                     without_xsect=True,
                     order_weight_fct = lambda n,*a : float(order<=n<=max_order))
        if order!=0:
            knlargs['order_weight_fct'] = lambda n,*a : float(order<=n<=max_order)
        di.extract_custom_knl(**knlargs,phasespace_curves=ne)
    elif choice=='cross sections':
        c=NC.NCMATComposer(plotlabel='bla')
        c.set_dyninfo_from_object('dummy',di)
        c.set_composition('dummy',di.atomData.description(False))
        c.set_density(1.0)
        c.plot_xsect(show_absorption=False,extra_cfg='comp=inelas',mode='wl',title=False)
    else:
        assert False

interactive(children=(IntSlider(value=0, description='order', max=50), IntSlider(value=0, description='max_ord…

In [8]:
#DONT LOOK AT THIS CODE, JUST HIDE IT AND PLAY WITH THE RESULTING WIDGET
_prev=[None]
@interact(debye_temp=(50.0,2000.0),atom_mass=(1.008,250.0),temperature=(1.0,2000),alpha_max=(1,500))
def show(debye_temp = 300,atom_mass=56.0,temperature=300.0,alpha_max=100):
    msd=NC.debyeIsotropicMSD(debye_temperature=debye_temp,mass=atom_mass,temperature=temperature)
    print('msd:',msd)
    import NCrystal.constants as c
    mn = c.const_neutron_mass_amu*c.constant_dalton2eVc2/c.constant_c**2
    k_alpha2qsq = c.constant_planck**2/(8*c.kPi**2*mn*c.constant_boltzmann)
    alpha = np.linspace(0.0,alpha_max,2000)
    import matplotlib.pyplot as plt
    if _prev[0]:
        plt.close(_prev[0])
    _prev[0]=plt.figure()
    def fact(i):
        return 1#fixme
        return 1 if i<=1 else i*fact(i=1)
    qsq = alpha*temperature/k_alpha2qsq
    twow = msd*qsq
    #plt.plot(twow)
    for n in [1,2,3,4,5]:
        y=np.exp(-twow)*(twow**n)/fact(n)
        plt.plot(alpha,y/np.trapz(x=alpha,y=y),label=f'n={n}')
    plt.suptitle("α-dependent factor of n'th term in Sjölander's expansion (normalised)",fontsize=6)
    plt.title("(β-dependency comes from Gn function)",fontsize=6)
    plt.grid()
    plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=300.0, description='debye_temp', max=2000.0, min=50.0), FloatSlider(va…